<center><h1>2. Vainilla CNNs</h1></center>

In [12]:
# Necessary modules
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D

In [25]:
# Building a vainilla CNN
vainilla_cnn = Sequential()
vainilla_cnn.add(Convolution2D(32, 3, 3,input_shape=(300,300,1)))
vainilla_cnn.add(Activation('relu'))
vainilla_cnn.add(MaxPooling2D(pool_size=(2, 2)))
vainilla_cnn.add(Convolution2D(64, 3, 3))
vainilla_cnn.add(Activation('relu'))
vainilla_cnn.add(Convolution2D(64, 3, 3))
vainilla_cnn.add(Activation('relu'))
vainilla_cnn.add(MaxPooling2D(pool_size=(2, 2)))
vainilla_cnn.add(Flatten())
vainilla_cnn.add(Dense(output_dim=100, input_dim=500))
vainilla_cnn.add(Activation("relu"))
vainilla_cnn.add(Dense(output_dim=200, input_dim=500))
vainilla_cnn.add(Activation("relu"))
vainilla_cnn.add(Dense(output_dim=1))
vainilla_cnn.add(Activation("sigmoid"))

In [26]:
# Compile
vainilla_cnn.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [28]:
# Train
# [FINISH ME]